# Bật GPU
"Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

In [1]:
!nvidia-smi

Tue Aug  4 00:09:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Cài đặt các gói cần thiết

In [3]:
%%shell
pip3 install transformers
git clone https://github.com/pytorch/fairseq
cd fairseq
pip3 install --editable ./

fatal: destination path 'fairseq' already exists and is not an empty directory.
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51kB 1.4MB/s 
  Running setup.py develop for fairseq


In [4]:
import torch
print(torch.__version__)

1.6.0+cu101


# Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive/')
root_dir = "/content/drive/My\ Drive/MasterThesis/CoLab/code_final"
src_dir = "%s/src" % root_dir
!ls $root_dir
!ls $src_dir
import sys
sys.path.append(src_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
cache	data	      model_stage2  src		TRAIN0.sh  TRAIN2.ipynb
corpus	model_stage1  output	    TEST.ipynb	TRAIN1.sh  TRAIN3.ipynb
bert_tokenize.py	 __init__.py	save_vocab_file.py
dictionary_with_bert.py  preprocess.py	transformer_with_pretrained_bert.py
gec_data_split.py	 __pycache__	translation_with_bert.py


# Evaluation

In [ ]:
!mkdir -p "/content/drive/My Drive/MasterThesis/CoLab/code_final/output"

## Evaluation

In [ ]:
!fairseq-generate "/content/drive/My Drive/MasterThesis/CoLab/code_final/data" -s 'input' -t 'target' \
	--fp16 \
	--user-dir "/content/drive/My Drive/MasterThesis/CoLab/code_final/src" --task translation_with_bert \
	--bert-model 'bert-base-multilingual-cased' \
	--gen-subset 'test' \
	--path "/content/drive/My Drive/MasterThesis/CoLab/code_final/model_stage2/checkpoint_best.pt" \
	--lenpen 1.0 \
	--beam 5 \
	--batch-size 256 \
  --remove-bpe \
  --num-workers 32 \
	> "/content/drive/My Drive/MasterThesis/CoLab/code_final/output/evaluation1.log"

In [14]:
!grep ^H "/content/drive/My Drive/MasterThesis/CoLab/code_final/output/evaluation1.log" \
 | cut -f3- \
 | sed 's/@@ //g' \
 > "/content/drive/My Drive/MasterThesis/CoLab/code_final/output/evaluation2.log.sys"

!grep ^T "/content/drive/My Drive/MasterThesis/CoLab/code_final/output/evaluation1.log" \
 | cut -f2- \
 | sed 's/@@ //g' \
 > "/content/drive/My Drive/MasterThesis/CoLab/code_final/output/evaluation2.log.ref"

In [15]:
!fairseq-score \
 --sys "/content/drive/My Drive/MasterThesis/CoLab/code_final/output/evaluation2.log.sys" \
 --ref "/content/drive/My Drive/MasterThesis/CoLab/code_final/output/evaluation2.log.ref"

2020-08-04 01:45:27.587384: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(ignore_case=False, order=4, ref='/content/drive/My Drive/MasterThesis/CoLab/code_final/output/evaluation2.log.ref', sacrebleu=False, sentence_bleu=False, sys='/content/drive/My Drive/MasterThesis/CoLab/code_final/output/evaluation2.log.sys')
BLEU4 = 92.37, 97.1/94.9/93.0/91.2 (BP=0.982, ratio=0.982, syslen=28876883, reflen=29397536)
